# Описание проекта

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

Данные геологоразведки трёх регионов находятся в файлах:

 - /datasets/geo_data_0.csv.
 - /datasets/geo_data_1.csv. 
 - /datasets/geo_data_2.csv.
 - id — уникальный идентификатор скважины;
 - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
 - product — объём запасов в скважине (тыс. баррелей).
 
**Условия задачи:**

 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
 - Бюджет на разработку скважин в регионе — 10 млрд рублей.
 - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
 - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
 
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 1. Загрузка и подготовка данных

Первым делом загрузим файлы и посмотрим, что они из себя представляют.

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно, чтобы все импорты были собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [62]:
import pandas as pd

data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [63]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [64]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [65]:
print(data_0.info())
print()
print(data_1.info())
print()
print(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memor

In [66]:
print(data_0['id'].duplicated().sum())
print(data_1['id'].duplicated().sum())
print(data_2['id'].duplicated().sum())

10
4
4


Каждый из трех датасетов содержит в себе по 100000 объектов, описанных пятью признаками. В качестве целевого признака для обучения модели принимаем признак `product`, описывающий объём запасов в скважине (тыс. баррелей), и будем использовать линейную регрессию (из-за малого количества параметров она не склонна к переобучению).
В данных присутствует незначительное количество дубликатов, что не повлияет на качество результата, поэтому оставим их без изменения.
Данные столбца `id` являются уникальными идентификаторами скважин и не несут ценной информации для обучения нашей модели.

<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка данных и их первичный анализ сделаны отлично!
</div>

# 2. Обучение и проверка модели

**2.1 Разбивка данных и масштабирование признаков**

Даннные будем разбивать на обучающую и валидационную выборки в соотношении 75:25.
А для достижения наилучшего результата для каждого региона будем использовать отдельную модель.

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Первый регион (0_REG)

In [68]:
target_0 = data_0['product']
features_0 = data_0.drop(['product', 'id'], axis=1)

# Стандартизируем признаки
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_0[numeric])
features_0[numeric] = scaler.transform(features_0[numeric])

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

Второй регион (1_REG)

In [69]:
target_1 = data_1['product']
features_1 = data_1.drop(['product', 'id'], axis=1)

# Стандартизируем признаки
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_1[numeric])
features_1[numeric] = scaler.transform(features_1[numeric])

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

Третий регион (2_REG)

In [70]:
target_2 = data_2['product']
features_2 = data_2.drop(['product', 'id'], axis=1)

# Стандартизируем признаки
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_2[numeric])
features_2[numeric] = scaler.transform(features_2[numeric])

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

<div class="alert alert-block alert-info">
<b>Совет: </b> Масштабировать признаки нужно было после разделения выборки на двы выборки. Обучать scaler нужно было только на тренировочной выборке.
</div>

**2.2 Обучение моделей**

Обучим модели и сделаем предсказания на валидационных выборках.

In [71]:
model_0_REG = LinearRegression()
model_0_REG.fit(features_train_0, target_train_0)
prediction_0 = model_0_REG.predict(features_valid_0)

model_1_REG = LinearRegression()
model_1_REG.fit(features_train_1, target_train_1)
prediction_1 = model_1_REG.predict(features_valid_1)

model_2_REG = LinearRegression()
model_2_REG.fit(features_train_2, target_train_2)
prediction_2 = model_2_REG.predict(features_valid_2)

In [72]:
print('Средний запас предсказанного сырья для 0_REG:', prediction_0.mean())
print('RMSE Линейной регрессии для 0_REG:', '\t', mean_squared_error(target_valid_0, prediction_0) ** 0.5)
print()
print('Средний запас предсказанного сырья для 1_REG:', prediction_1.mean())
print('RMSE Линейной регрессии для 1_REG:', '\t', mean_squared_error(target_valid_1, prediction_1) ** 0.5)
print()
print('Средний запас предсказанного сырья для 2_REG:', prediction_2.mean())
print('RMSE Линейной регрессии для 2_REG:', '\t', mean_squared_error(target_valid_2, prediction_2) ** 0.5)

Средний запас предсказанного сырья для 0_REG: 92.59256778438038
RMSE Линейной регрессии для 0_REG: 	 37.5794217150813

Средний запас предсказанного сырья для 1_REG: 68.72854689544602
RMSE Линейной регрессии для 1_REG: 	 0.8930992867756161

Средний запас предсказанного сырья для 2_REG: 94.96504596800489
RMSE Линейной регрессии для 2_REG: 	 40.02970873393434


В глаза сразу бросается огромный разброс в значении ошибок. Наименьшее значение RMSE у второго региона. Это может быть связано с разницей данных - возможно во втором регионе данные попроще и модель прекрасно с ними справляется, в то время, как в 1-ом и 3-м регионах данные имеют более сложую структуру.
Значение среднего запаса предсказанного сырья так же выделяется у второго региона (68.72 тыс.баррелей) и заментно ниже, чем у 1-го и 3-го.

<div class="alert alert-block alert-success">
<b>Успех:</b> Обучение моделей сделано верно!
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> На протяжении всего проекта происходит работа с 3мя одинаковыми таблицами. В связи с этим можно было использовать функции и/или циклы, чтобы избежать копирование кода.
<br> Зачем его нужно избегать? Потому что очень часто оно приводит к ошибкам (например, забыл поменять одну из переменных). Впоследствии такие ошибки очень трудно найти.
</div>

# 3. Подготовка к расчёту прибыли

In [73]:
budget = 10_000_000_000 # Бюджет на разработку скважин в регионе.
barrel_price = 450_000 # Цена за 1 тыс.баррелей
well = 500 # Количчество скважин

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было использовать только большие буквы в названиях константных переменных: "WELL".
</div>

Примем, что по всем скважинам расход на разработку одинаковый.

In [74]:
one_well_price = budget / well
print('Стоимость одной скважины:', one_well_price, 'руб.')

Стоимость одной скважины: 20000000.0 руб.


Определим достаточный объём сырья разработки новой скважины.

In [75]:
amount_of_oil = one_well_price / barrel_price
amount_of_oil

44.44444444444444

<div class="alert alert-block alert-info">
<b>Совет: </b> Достаточный объем сырья посчитан неверно: разрабатывается только 200 скважин, а не все 500.
</div>

Как мы можем видеть, минимальный объем нефти, необходимый для безубыточной разработки новой нефти составляет 45 тыс.баррелей, что значительно меньше средних значений во всех трех имеющихся у нас регионах.

# 4. Расчёт прибыли и рисков 

Создадим объекты *Series* из предсказаний моделей, что бы указать правильные индексы.

In [76]:
prediction_0 = pd.Series(prediction_0, index=target_valid_0.index)
prediction_1 = pd.Series(prediction_1, index=target_valid_1.index)
prediction_2 = pd.Series(prediction_2, index=target_valid_2.index)

Напишим функцию для расчета прибыли по по определенному количеству скважин.

In [77]:
def profit(target, predictions, count):
    pred_sorted = predictions.sort_values(ascending=False)
    selected_top = target[pred_sorted.index][:count]
    income = selected_top.sum() * barrel_price
    profit = income - budget
    return profit

print('Прибыль с первого региона:', profit(target_valid_0, prediction_0, 200))
print('Прибыль со второго региона:', profit(target_valid_1, prediction_1, 200))
print('Прибыль с третьего региона:', profit(target_valid_2, prediction_2, 200))

Прибыль с первого региона: 3320826043.1398525
Прибыль со второго региона: 2415086696.681511
Прибыль с третьего региона: 2710349963.5998325


<div class="alert alert-block alert-success">
<b>Успех:</b> Функция расчета прибыли определна абсолютно верно!
</div>

# 5. Подсчет рисков и прибыли для каждого региона

**Первый регион**

In [78]:
import numpy as np

state = np.random.RandomState(12345)
values_0 = []
for i in range(1000):
    target_subsamples = target_valid_0.sample(n=500, replace=True, random_state=state)
    probs_subsamples = prediction_0[target_subsamples.index]
    values_0.append(profit(target_subsamples, probs_subsamples, 200))
    
values_0 = pd.Series(values_0)
negative_profit_chance = (values_0 < 0).mean()
lower_0 = values_0.quantile(0.025)
upeer_0 = values_0.quantile(0.975)

mean_0 = values_0.mean()

print('Средняя выручка первого региона:', mean_0)
print('Доверительный интервал первого региона:','от', lower_0, 'до', upeer_0)
print('Риск убытка для первого региона составляет {:.1%}'.format(negative_profit_chance))

Средняя выручка первого региона: 425938526.9105923
Доверительный интервал первого региона: от -102090094.83793654 до 947976353.358369
Риск убытка для первого региона составляет 6.0%


**Второй регион**

In [79]:
values_1 = []
for i in range(1000):
    target_subsamples = target_valid_1.sample(n=500, replace=True, random_state=state)
    probs_subsamples = prediction_1[target_subsamples.index]
    values_1.append(profit(target_subsamples, probs_subsamples, 200))
    
values_1 = pd.Series(values_1)
negative_profit_chance = (values_1 < 0).mean()
lower_1 = values_1.quantile(0.025)
upeer_1 = values_1.quantile(0.975)

mean_1 = values_1.mean()

print('Средняя выручка второго региона:', mean_1)
print('Доверительный интервал второго региона:','от', lower_1, 'до', upeer_1)
print('Риск убытка для второго региона составляет {:.1%}'.format(negative_profit_chance))

Средняя выручка второго региона: 518259493.69732493
Доверительный интервал второго региона: от 128123231.43308629 до 953612982.0669085
Риск убытка для второго региона составляет 0.3%


**Третий регион**

In [80]:
values_2 = []
for i in range(1000):
    target_subsamples = target_valid_2.sample(n=500, replace=True, random_state=state)
    probs_subsamples = prediction_2[target_subsamples.index]
    values_2.append(profit(target_subsamples, probs_subsamples, 200))
    
values_2 = pd.Series(values_2)
negative_profit_chance = (values_2 < 0).mean()
lower_2 = values_2.quantile(0.025)
upeer_2 = values_2.quantile(0.975)

mean_2 = values_2.mean()

print('Средняя выручка третьего региона:', mean_2)
print('Доверительный интервал третьего региона:','от', lower_2, 'до', upeer_2)
print('Риск убытка для третьего региона составляет {:.1%}'.format(negative_profit_chance))

Средняя выручка третьего региона: 420194005.34405005
Доверительный интервал третьего региона: от -115852609.16001143 до 989629939.844574
Риск убытка для третьего региона составляет 6.2%


<div class="alert alert-block alert-success">
<b>Успех:</b> Все значения посчитаны верно, молодец!
</div>

In [81]:
prof_0 = profit(target_valid_0, prediction_0, 200)
prof_1 = profit(target_valid_1, prediction_1, 200)

print('Разница между прибылью первого и второго региона: {:.0%}'.format(1 - prof_1 / prof_0))

Разница между прибылью первого и второго региона: 27%


<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было собрать все результаты в одну таблицу.
</div>

### Вывод

В результате проделанной нами работы мы можем видеть заметные отличия второго региона от двух оставшихся. В нем содержится заметно меньшие запасы сырья в сравнении с первым и третьим, а так же он на 27% менее прибыльный, чем первый (который является наиболее прибыльным по расчетам). Несмотря на все это, риски претерпеть убытки при разработки скважин с последующей добычей нефти во втором регионе значительно меньше аналогов, а с вероятностью 95% выручка может составлять не менее 516 млн.руб. что является решающим фактором по выбору региона для добычи. 
Исходя из всего вышеперечисленного, заключаем, что второй регион является наиболее прибыльным для разработки месторождений и последующей добычей сырья. :-)

<div class="alert alert-block alert-success">
<b>Успех:</b> Регион выбран абсолютно верно, молодец!
</div>

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован